In [212]:
import numpy as np
import pandas as pd
import itertools

from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge,Lasso

import statsmodels.api as sm
from scipy import stats

from tqdm import tqdm #Importing tqdm for the progress bar

Ref:
1. [full subset-search](https://xavierbourretsicotte.github.io/subset_selection.html)


## Introductory analyses

In [4]:
data = pd.read_csv('../data/housing-p.data.txt',sep=' ',header=0)

In [5]:
X = data.iloc[:,0:-1]
y = data.iloc[:,-1]

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=156,shuffle=False)

In [8]:
X_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.01301,35.0,1.52,0,0.442,7.241,49.3,7.0379,1,284,15.5,394.74,5.49
1,9.39063,0.0,18.10,0,0.740,5.627,93.9,1.8172,24,666,20.2,396.90,22.88
2,0.54011,20.0,3.97,0,0.647,7.203,81.8,2.1121,5,264,13.0,392.80,9.59
3,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68
4,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14


In [60]:
X_test = sm.add_constant(X_test)

In [12]:
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     75.43
Date:                Fri, 08 Nov 2019   Prob (F-statistic):           3.03e-91
Time:                        16:48:01   Log-Likelihood:                -1029.2
No. Observations:                 350   AIC:                             2086.
Df Residuals:                     336   BIC:                             2140.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.4003      6.174      6.381      0.0

/Users/jeremydai/anaconda/envs/p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## Report estimated coefficients, their standard errors, and statistical significance 

## Cross-validation and regularization

In [99]:
del X_train['const'] 

In [189]:
del X_test['const'] 

In [100]:
size = int(len(y_train)/5)

In [156]:
RSS_list, R_squared_list = [],[]
for i in range(5):
    test_index = np.arange(i*size,(i+1)*size)
    train_index = [x for x in np.arange(0,350) if x not in test_index]
    x_tr = X_train.iloc[train_index]
    x_te = X_train.iloc[test_index]
    y_tr = y_train.iloc[train_index]
    y_te = y_train.iloc[test_index]
    model = LinearRegression()
    model.fit(x_tr, y_tr)
    RSS_list.append(mean_squared_error(y_te,model.predict(x_te)) * size)
    R_squared_list.append(model.score(x_te,y_te)) 

In [158]:
RSS_list,R_squared_list

([1827.6501497550853,
  1169.2532911104697,
  2524.6743776378926,
  1388.4699273684323,
  1404.6408627703004],
 [0.7225827346061061,
  0.7905611510389149,
  0.5381108326374417,
  0.7058821013155587,
  0.777761071116262])

In [161]:
np.mean(RSS_list),np.mean(R_squared_list)

(1662.937721728436, 0.7069795781428567)

### Perform full subset-search

In [162]:
def fit_linear_reg(X,y):
    #Fit linear regression model and return RSS and R squared values
    rss_list, r_squared_list = [],[]
    for i in range(5):
        test_index = np.arange(i*size,(i+1)*size)
        train_index = [x for x in np.arange(0,350) if x not in test_index]
        x_tr = X.iloc[train_index]
        x_te = X.iloc[test_index]
        y_tr = y.iloc[train_index]
        y_te = y.iloc[test_index]
        model = LinearRegression()
        model.fit(x_tr, y_tr)
        rss_list.append(mean_squared_error(y_te,model.predict(x_te)) * size)
        r_squared_list.append(model.score(x_te,y_te)) 
    return np.mean(rss_list),np.mean(r_squared_list)

In [163]:
#Initialization variables
K = len(X_train.columns)
RSS_list, R_squared_list, feature_list = [],[],[]
numb_features = []

In [164]:
#Looping over k = 1 to K features in X_train
for k in tqdm(range(K)):
    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X_train.columns,k+1):
        tmp_result = fit_linear_reg(X_train[list(combo)],y_train)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})

100%|██████████| 13/13 [04:42<00:00, 21.70s/it]


In [165]:
df.groupby('numb_features').apply(len)

numb_features
1       13
2       78
3      286
4      715
5     1287
6     1716
7     1716
8     1287
9      715
10     286
11      78
12      13
13       1
dtype: int64

In [166]:
df_min = df[df['RSS'] == df.groupby('numb_features')['RSS'].transform(min)]
df_max = df[df['R_squared'] == df.groupby('numb_features')['R_squared'].transform(max)]
display(df_min.head(3))
display(df_max.head(3))

,numb_features,RSS,R_squared,features
12,1,2554.516940,0.555532,"(LSTAT,)"
69,2,2138.124258,0.625233,"(RM, LSTAT)"
340,3,1922.073981,0.659838,"(RM, PTRATIO, LSTAT)"


,numb_features,RSS,R_squared,features
12,1,2554.516940,0.555532,"(LSTAT,)"
69,2,2138.124258,0.625233,"(RM, LSTAT)"
340,3,1922.073981,0.659838,"(RM, PTRATIO, LSTAT)"


### Apply ridge regression

In [201]:
def fit_ridge_reg(X,y,C):
    #Fit ridge regression model and return RSS and R squared values
    rss_list, r_squared_list = [],[]
    for i in range(5):
        test_index = np.arange(i*size,(i+1)*size)
        train_index = [x for x in np.arange(0,350) if x not in test_index]
        x_tr = X.iloc[train_index]
        x_te = X.iloc[test_index]
        y_tr = y.iloc[train_index]
        y_te = y.iloc[test_index]
        model = Ridge(alpha=C,fit_intercept=True, normalize=False)
        model.fit(x_tr, y_tr)
        rss_list.append(mean_squared_error(y_te,model.predict(x_te)) * size)
        r_squared_list.append(model.score(x_te,y_te)) 
    return np.mean(rss_list),np.mean(r_squared_list)

In [202]:
RSS_list, R_squared_list, C_list = [],[],[]
#Looping over alphas=[1e-3, 1e-2, 1e-1, 1]
alphas=[1e-3, 1e-2, 1e-1, 1]
for i in tqdm(range(len(alphas))):
    C = alphas[i]
    tmp_result = fit_ridge_reg(X_train,y_train,C)   #Store temp result 
    RSS_list.append(tmp_result[0])                  #Append lists
    R_squared_list.append(tmp_result[1])
    C_list.append(C) 

#Store in DataFrame
df = pd.DataFrame({'C': C_list,'RSS': RSS_list, 'R_squared':R_squared_list})

100%|██████████| 4/4 [00:00<00:00, 26.97it/s]


In [205]:
df[df.RSS == min(df.RSS)]

,C,RSS,R_squared
2,0.1,1662.427963,0.707042


In [208]:
model = Ridge(alpha=0.1,fit_intercept=True, normalize=False)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7181578088398777

In [211]:
model.intercept_

38.32995124632765

In [209]:
model.coef_

array([-1.12417348e-01,  6.40352034e-02,  1.88755809e-02,  1.47103397e+00,
       -1.54660207e+01,  3.36485169e+00, -6.58189875e-03, -1.61813600e+00,
        2.94848752e-01, -1.38844361e-02, -8.25733936e-01,  6.38936471e-03,
       -5.24396070e-01])

### Apply lasso regression

In [213]:
def fit_lasso_reg(X,y,C):
    #Fit ridge regression model and return RSS and R squared values
    rss_list, r_squared_list = [],[]
    for i in range(5):
        test_index = np.arange(i*size,(i+1)*size)
        train_index = [x for x in np.arange(0,350) if x not in test_index]
        x_tr = X.iloc[train_index]
        x_te = X.iloc[test_index]
        y_tr = y.iloc[train_index]
        y_te = y.iloc[test_index]
        model = Lasso(alpha=C,fit_intercept=True, normalize=False)
        model.fit(x_tr, y_tr)
        rss_list.append(mean_squared_error(y_te,model.predict(x_te)) * size)
        r_squared_list.append(model.score(x_te,y_te)) 
    return np.mean(rss_list),np.mean(r_squared_list)

In [214]:
RSS_list, R_squared_list, C_list = [],[],[]
#Looping over alphas=[1e-3, 1e-2, 1e-1, 1]
alphas=[1e-3, 1e-2, 1e-1, 1]
for i in tqdm(range(len(alphas))):
    C = alphas[i]
    tmp_result = fit_lasso_reg(X_train,y_train,C)   #Store temp result 
    RSS_list.append(tmp_result[0])                  #Append lists
    R_squared_list.append(tmp_result[1])
    C_list.append(C) 

#Store in DataFrame
df = pd.DataFrame({'C': C_list,'RSS': RSS_list, 'R_squared':R_squared_list})

100%|██████████| 4/4 [00:00<00:00, 24.08it/s]


In [217]:
df[df.RSS == min(df.RSS)]

,C,RSS,R_squared
0,0.001,1663.102638,0.706945


In [219]:
model = Lasso(alpha=0.001,fit_intercept=True, normalize=False)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7183631397822813

In [220]:
print(clf.coef_)
print(clf.intercept_)

[-1.07633153e-01  6.55659731e-02 -7.74108429e-03  1.37940315e+00
 -8.46025746e+00  3.37750503e+00 -1.18475496e-02 -1.50930496e+00
  2.75814143e-01 -1.43614837e-02 -7.52768028e-01  6.72211300e-03
 -5.38298821e-01]
33.683198219160374
